In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import pandas as pd
import glob
import pickle
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3

import matplotlib

matplotlib.pyplot.set_loglevel(level="warning")

from nmf_ik.utils import (
    dict_to_nparray_pose,
    save_file,
    interpolate_signal,
    calculate_nmf_size,
)
from nmf_ik.data import BOUNDS, NMF_SIZE, NMF_TEMPLATE, INITIAL_ANGLES
from nmf_ik.leg_inverse_kinematics import LegInverseKinematics
from nmf_ik.kinematic_chain import KinematicChainYPR
from nmf_ik.visualization import plot_3d_points, animate_3d_points

import sys

aligned_pose_path = Path("../data/3d_pose_aligned/df3dpp_aligned.pkl")
# Next two data come from the paper
aligned_pose_pp = pd.read_pickle(aligned_pose_path)
output_path = aligned_pose_path.parent.parent / "inverse_kinematics"


ANGLES = [
    "ThC_yaw",
    "ThC_pitch",
    "ThC_roll",
    "CTr_pitch",
    "CTr_roll",
    "FTi_pitch",
    "TiTa_pitch",
]

In [ ]:
# Aligned pose now in a nested dictionary format. -> dictionary[leg_name][key_point_name][raw_pos_aligned]
# My package is using numpy arrays, so we convert the pose dictionary into a compatible format

print(aligned_pose_pp.keys())
converted_aligned_pose = {}

for leg_name, leg_pose_data in aligned_pose_pp.items():
    if leg_name in ["meta", "swing_stance_time"]:
        continue
    #     print(leg_pose_data.keys())
    converted_aligned_pose[leg_name] = dict_to_nparray_pose(
        leg_pose_data, claw_is_end_effector=True
    )

In [ ]:
print(converted_aligned_pose.keys())

converted_aligned_pose["RF_leg"].shape  # timestep, key points, coordinates (x,y,z)

In [ ]:
# let's save the new dictionary
export_path_seqikpy_format_aligned_pos = output_path / "pose3d_aligned.pkl"
export_path_seqikpy_format_aligned_pos.parent.mkdir(parents=True, exist_ok=True)
save_file(export_path_seqikpy_format_aligned_pos, converted_aligned_pose)

In [ ]:
# We will use these configs for the 3D plotting
KEY_POINTS_DICT = {
    "LH": (np.arange(25, 30), "solid"),
    "LM": (np.arange(20, 25), "solid"),
    "LF": (np.arange(15, 20), "solid"),
    "RH": (np.arange(10, 15), "solid"),
    "RM": (np.arange(5, 10), "solid"),
    "RF": (np.arange(0, 5), "solid"),
}

KEY_POINTS_DICT2 = {
    "LH": (np.arange(45, 54), ":"),
    "LM": (np.arange(36, 45), ":"),
    "LF": (np.arange(27, 36), ":"),
    "RH": (np.arange(18, 27), ":"),
    "RM": (np.arange(9, 18), ":"),
    "RF": (np.arange(0, 9), ":"),
}

In [ ]:
# Plot the aligned pose
points_aligned_all = np.concatenate(
    (
        converted_aligned_pose["RF_leg"],
        converted_aligned_pose["RM_leg"],
        converted_aligned_pose["RH_leg"],
        converted_aligned_pose["LF_leg"],
        converted_aligned_pose["LM_leg"],
        converted_aligned_pose["LH_leg"],
    ),
    axis=1,
)

ax_limits = [-2.0, 2.0]

aligned_pose_export_path = output_path / "viz/aligned_pose"
aligned_pose_export_path.mkdir(parents=True, exist_ok=True)
for t in range(0, points_aligned_all.shape[0], 1):
    for azim, elev in [(0, 0), (90, 0), (0, 90)]:
        fig = plt.figure(figsize=(12, 12))
        ax3d = fig.add_subplot(111, projection="3d")
        ax3d.view_init(azim=azim, elev=elev)

        ax3d.set_xlabel("x")
        ax3d.set_ylabel("y")
        ax3d.set_zlabel("z")

        plot_3d_points(
            ax3d,
            points_aligned_all,
            KEY_POINTS_DICT,
            label_prefix="Aligned pose ",
            t=t,
            fix_lim=True,
            export_path=aligned_pose_export_path / f"azim_{azim}_elev_{elev}_{t}.png",
        )
        plt.close()

In [ ]:
# Now we converted the aligned pose from nested dictionary to numpy arrays.
# Let's start using seq_ik

# Step 1: alignment - already done using DF3DPP so we can skip

# Step 2. calculation of joint angles
# We are defining a set of initial seeds for the IK optimization

INITIAL_ANGLES_LOCOMOTION = {
    "RF": {
        # Base ThC yaw pitch CTr pitch
        "stage_1": np.array([0.0, 0.45, -0.07, -2.14]),
        # Base ThC yaw pitch roll CTr pitch CTr roll
        "stage_2": np.array([0.0, 0.45, -0.07, -0.32, -2.14, 1.4]),
        # Base ThC yaw pitch roll CTr pitch CTr roll FTi pitch
        "stage_3": np.array([0.0, 0.45, -0.07, -0.32, -2.14, -1.25, 1.48, 0.0]),
        # Base ThC yaw pitch roll CTr pitch CTr roll FTi pitch TiTa pitch
        "stage_4": np.array([0.0, 0.45, -0.07, -0.32, -2.14, -1.25, 1.48, 0.0, 0.0]),
    },
    "LF": {
        "stage_1": np.array([0.0, -0.45, -0.07, -2.14]),
        "stage_2": np.array([0.0, -0.45, -0.07, 0.32, -2.14, 1.4]),
        "stage_3": np.array([0.0, -0.45, -0.07, 0.32, -2.14, 1.25, 1.48, 0.0]),
        "stage_4": np.array([0.0, -0.45, -0.07, 0.32, -2.14, 1.25, 1.48, 0.0, 0.0]),
    },
    "RM": {
        "stage_1": np.array([0.0, 0.45, 0.37, -2.14]),
        "stage_2": np.array([0.0, 0.45, 0.37, -0.32, -2.14, 1.4]),
        "stage_3": np.array([0.0, 0.45, 0.37, -0.32, -2.14, -1.25, 1.48, 0.0]),
        "stage_4": np.array([0.0, 0.45, 0.37, -0.32, -2.14, -1.25, 1.48, 0.0, 0.0]),
    },
    "LM": {
        "stage_1": np.array([0.0, -0.45, 0.37, -2.14]),
        "stage_2": np.array([0.0, -0.45, 0.37, 0.32, -2.14, 1.4]),
        "stage_3": np.array([0.0, -0.45, 0.37, 0.32, -2.14, 1.25, 1.48, 0.0]),
        "stage_4": np.array([0.0, -0.45, 0.37, 0.32, -2.14, 1.25, 1.48, 0.0, 0.0]),
    },
    "RH": {
        "stage_1": np.array([0.0, 0.45, 0.07, -2.14]),
        "stage_2": np.array([0.0, 0.45, 0.07, -0.32, -2.14, 1.4]),
        "stage_3": np.array([0.0, 0.45, 0.07, -0.32, -2.14, -1.25, 1.48, 0.0]),
        "stage_4": np.array([0.0, 0.45, 0.07, -0.32, -2.14, -1.25, 1.48, 0.0, 0.0]),
    },
    "LH": {
        "stage_1": np.array([0.0, -0.45, 0.07, -2.14]),
        "stage_2": np.array([0.0, -0.45, 0.07, 0.32, -2.14, 1.4]),
        "stage_3": np.array([0.0, -0.45, 0.07, 0.32, -2.14, 1.25, 1.48, 0.0]),
        "stage_4": np.array([0.0, -0.45, 0.07, 0.32, -2.14, 1.25, 1.48, 0.0, 0.0]),
    },
    "head": np.array([0, -0.17, 0]),  #  none, roll, pitch, yaw
}

# We define a template to create the kinematic chain
# The length of chain comes from the size calculated from the template

TEMPLATE_NMF_LOCOMOTION = {
    "RF_Coxa": np.array([0.35, -0.27, 0.400]),
    "RF_Femur": np.array([0.35, -0.27, -0.025]),
    "RF_Tibia": np.array([0.35, -0.27, -0.731]),
    "RF_Tarsus": np.array([0.35, -0.27, -1.249]),
    "RF_Claw": np.array([0.35, -0.27, -1.912]),
    "LF_Coxa": np.array([0.35, 0.27, 0.400]),
    "LF_Femur": np.array([0.35, 0.27, -0.025]),
    "LF_Tibia": np.array([0.35, 0.27, -0.731]),
    "LF_Tarsus": np.array([0.35, 0.27, -1.249]),
    "LF_Claw": np.array([0.35, 0.27, -1.912]),
    "RM_Coxa": np.array([0, -0.125, 0]),
    "RM_Femur": np.array([0, -0.125, -0.182]),
    "RM_Tibia": np.array([0, -0.125, -0.965]),
    "RM_Tarsus": np.array([0, -0.125, -1.633]),
    "RM_Claw": np.array([0, -0.125, -2.328]),
    "LM_Coxa": np.array([0, 0.125, 0]),
    "LM_Femur": np.array([0, 0.125, -0.182]),
    "LM_Tibia": np.array([0, 0.125, -0.965]),
    "LM_Tarsus": np.array([0, 0.125, -1.633]),
    "LM_Claw": np.array([0, 0.125, -2.328]),
    "RH_Coxa": np.array([-0.215, -0.087, -0.073]),
    "RH_Femur": np.array([-0.215, -0.087, -0.272]),
    "RH_Tibia": np.array([-0.215, -0.087, -1.108]),
    "RH_Tarsus": np.array([-0.215, -0.087, -1.793]),
    "RH_Claw": np.array([-0.215, -0.087, -2.588]),
    "LH_Coxa": np.array([-0.215, 0.087, -0.073]),
    "LH_Femur": np.array([-0.215, 0.087, -0.272]),
    "LH_Tibia": np.array([-0.215, 0.087, -1.108]),
    "LH_Tarsus": np.array([-0.215, 0.087, -1.793]),
    "LH_Claw": np.array([-0.215, 0.087, -2.588]),
}

# We determine the bounds for each joint DOF
BOUNDS_LOCOMOTION = {
    "RF_ThC_yaw": (np.deg2rad(-45), np.deg2rad(45)),
    "RF_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "RF_ThC_roll": (np.deg2rad(-135), np.deg2rad(10)),
    "RF_CTr_pitch": (np.deg2rad(-270), np.deg2rad(10)),
    "RF_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "RF_CTr_roll": (np.deg2rad(-180), np.deg2rad(90)),
    "RF_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "RM_ThC_yaw": (np.deg2rad(-45), np.deg2rad(45)),
    "RM_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "RM_ThC_roll": (np.deg2rad(-180), np.deg2rad(10)),
    "RM_CTr_pitch": (np.deg2rad(-270), np.deg2rad(10)),
    "RM_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "RM_CTr_roll": (np.deg2rad(-90), np.deg2rad(90)),
    "RM_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "RH_ThC_yaw": (np.deg2rad(-45), np.deg2rad(45)),
    "RH_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "RH_ThC_roll": (np.deg2rad(-180), np.deg2rad(10)),
    "RH_CTr_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "RH_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "RH_CTr_roll": (np.deg2rad(-90), np.deg2rad(90)),
    "RH_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LF_ThC_yaw": (np.deg2rad(-45), np.deg2rad(45)),
    "LF_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "LF_ThC_roll": (np.deg2rad(-10), np.deg2rad(90)),
    "LF_CTr_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LF_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "LF_CTr_roll": (np.deg2rad(-90), np.deg2rad(180)),
    "LF_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LM_ThC_yaw": (np.deg2rad(-45), np.deg2rad(90)),
    "LM_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "LM_ThC_roll": (np.deg2rad(-10), np.deg2rad(180)),
    "LM_CTr_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LM_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "LM_CTr_roll": (np.deg2rad(-90), np.deg2rad(90)),
    "LM_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LH_ThC_yaw": (np.deg2rad(-45), np.deg2rad(90)),
    "LH_ThC_pitch": (np.deg2rad(-10), np.deg2rad(90)),
    "LH_ThC_roll": (np.deg2rad(-10), np.deg2rad(180)),
    "LH_CTr_pitch": (np.deg2rad(-180), np.deg2rad(10)),
    "LH_FTi_pitch": (np.deg2rad(-10), np.deg2rad(180)),
    "LH_CTr_roll": (np.deg2rad(-90), np.deg2rad(90)),
    "LH_TiTa_pitch": (np.deg2rad(-180), np.deg2rad(10)),
}

# In the end using bounds that are very large as what matters in the end is reproducing the movements
for key, val in BOUNDS_LOCOMOTION.items():
    BOUNDS_LOCOMOTION[key] = (-np.pi, np.pi)


# Calculate the size from the TEMPLATE_NMF_LOCOMOTION above
NMF_SIZE = calculate_nmf_size(TEMPLATE_NMF_LOCOMOTION)

In [ ]:
# Double check to see how size differs from the template
nmf_size_from_aligned_pos = {}

for leg, leg_dict in aligned_pose_pp.items():
    if leg in ["meta", "swing_stance_time"]:
        continue
    for segment in leg_dict:
        try:
            print(
                "DF3DPP: ",
                leg,
                segment,
                leg_dict[segment]["mean_length"],
                "TEMPLATE: ",
                NMF_SIZE[f"{leg[:2]}_{segment}"],
            )
            nmf_size_from_aligned_pos[f"{leg[:2]}_{segment}"] = leg_dict[segment][
                "mean_length"
            ]
        except:
            print(
                "DF3DPP: ",
                leg,
                segment,
                leg_dict[segment]["total_length"],
                "TEMPLATE: ",
                NMF_SIZE[f"{leg[:2]}"],
            )

In [ ]:
# Let's run the seq IK..
kinematic_chain_class = KinematicChainYPR(
    bounds_dof=BOUNDS_LOCOMOTION,
    # Using the size comes from the alignment.
    nmf_size=nmf_size_from_aligned_pos,
    # nmf_size=NMF_SIZE,
)

class_seq_ik = LegInverseKinematics(
    aligned_pos=converted_aligned_pose,
    kinematic_chain_class=kinematic_chain_class,
    initial_angles=INITIAL_ANGLES_LOCOMOTION,
)

# This will save two separate files:
# `leg_joint_angles.pkl` -> pickle file that containes the leg joint angles
# `forward_kinematics.pkl` -> 3D position of the legs construction from the calculated leg joint angles
# The latter is useful for debugging, visualization...
leg_joint_angles, forward_kinematics = class_seq_ik.run_ik_and_fk(
    export_path=None
)  # where the results will be saved
leg_joint_angles["meta"] = aligned_pose_pp["meta"]
leg_joint_angles["swing_stance_time"] = aligned_pose_pp["swing_stance_time"]
forward_kinematics["meta"] = aligned_pose_pp["meta"]
forward_kinematics["swing_stance_time"] = aligned_pose_pp["swing_stance_time"]
# save both as pkl files
with open(output_path / "leg_joint_angles.pkl", "wb") as f:
    pickle.dump(leg_joint_angles, f)
with open(output_path / "forward_kinematics.pkl", "wb") as f:
    pickle.dump(forward_kinematics, f)

In [ ]:
# Let's plot the joint angles for all six legs
plt.style.use("default")
fig, axs = plt.subplots(3, 2, figsize=(9, 6), dpi=100)

time = (
    np.arange(len(leg_joint_angles["Angle_LF_CTr_pitch"]))
    * aligned_pose_pp["meta"]["timestep"]
)

axs = axs.flatten()
for angle_name in ANGLES:
    for i, leg_name in enumerate(["RF", "LF", "RM", "LM", "RH", "LH"]):
        axs[i].plot(
            time,
            np.rad2deg(leg_joint_angles[f"Angle_{leg_name}_{angle_name}"]),
            label=angle_name,
            lw=2,
        )
        axs[i].set_ylabel(leg_name)
        axs[i].set_ylim(-180, 180)
        axs[i].set_yticks([-180, -90, 0, 90, 180])


axs[-1].set_xlabel("Time (s)")
axs[-2].set_xlabel("Time (s)")

axs[1].legend(bbox_to_anchor=(1.1, 1), frameon=False)

plt.suptitle("Joint angles [deg]")
plt.tight_layout()

fig.savefig(output_path / "viz/joint_angles.png", bbox_inches="tight")
plt.show()

In [ ]:
# Files saved automatically after the IK-FK process above.
pose3d = aligned_pose_path
forward_kinematics_path = output_path / "forward_kinematics.pkl"
forward_kinematics_path.parent.mkdir(parents=True, exist_ok=True)

with open(pose3d, "rb") as f:
    aligned_pose = pickle.load(f)
with open(forward_kinematics_path, "rb") as f:
    forward_kin = pickle.load(f)

points_aligned_all = np.concatenate(
    (
        converted_aligned_pose["RF_leg"],
        converted_aligned_pose["RM_leg"],
        converted_aligned_pose["RH_leg"],
        converted_aligned_pose["LF_leg"],
        converted_aligned_pose["LM_leg"],
        converted_aligned_pose["LH_leg"],
    ),
    axis=1,
)

points_fk = np.concatenate(
    (
        forward_kin["RF_leg"],
        forward_kin["RM_leg"],
        forward_kin["RH_leg"],
        forward_kin["LF_leg"],
        forward_kin["LM_leg"],
        forward_kin["LH_leg"],
    ),
    axis=1,
)

export_path_ik_fk_comparison = output_path / "viz/ik_fk_comparison"
export_path_ik_fk_comparison.mkdir(parents=True, exist_ok=True)


for t in range(0, points_aligned_all.shape[0], 1):
    for azim, elev in [(0, 0), (90, 0), (0, 90)]:
        fig = plt.figure(figsize=(12, 12))
        ax3d = fig.add_subplot(111, projection="3d")
        ax3d.view_init(azim=azim, elev=elev)

        ax3d.set_xlabel("x")
        ax3d.set_ylabel("y")
        ax3d.set_zlabel("z")

        plot_3d_points(
            ax3d,
            points_aligned_all,
            KEY_POINTS_DICT,
            label_prefix="Aligned pose ",
            t=t,
            fix_lim=False,
            export_path=None,
        )
        plot_3d_points(
            ax3d,
            points_fk,
            KEY_POINTS_DICT2,
            label_prefix="FK ",
            t=t,
            fix_lim=False,
            export_path=None,
        )
        ax3d.set_xlim([-2.0, 2.0])
        ax3d.set_ylim([-2.0, 2.0])
        ax3d.set_zlim([-2.0, 2.0])

        fig.savefig(
            str(export_path_ik_fk_comparison / f"azim_{azim}_elev_{elev}_{t}.png"),
            bbox_inches="tight",
        )
        plt.close()